In [ ]:
%%capture
!pip install --no-index --find-links ../input/huggingface-datasets datasets -qq

In [ ]:
!pip install wandb

In [ ]:
!pip install -U adapter-transformers

In [ ]:
import pandas as pd
import numpy as np
import os
import json
import collections

from pathlib import Path
from typing import List, Dict, Optional, Union
from pydantic import BaseModel

import datasets
from transformers.trainer_utils import set_seed
from transformers import (AutoTokenizer, PreTrainedTokenizerFast,
                          AutoModelForQuestionAnswering, TrainingArguments,
                          Trainer, default_data_collator, DataCollatorWithPadding)
import wandb

In [ ]:
class Config:
    # Path
    model_path: Path = '../input/huggingface-question-answering-models/en/xlm-roberta-base-squad2'
    test_path: Path = ''
    output_dir: str = './'
    
    # Base
    model_name: str = 'bert-base-multilingual-cased'
    version: str = 'hindi-A'
    seed: int = 42
    test_size: float = 0.1
    debug=False
    
    # Tokenizer
    max_length: int = 256
    doc_stride: int = 128
    
    # Trainer
    batch_size: int = 32
    learning_rate: float = 2e-5
    warmup_ratio: float = 0.1
    gradient_accumulation_steps: int = 4
    num_train_epochs: int = 2
    weight_decay: float = 0.01
    
    # Postprocess
    n_best_size: int = 20
    max_answer_length: int = 30
    squad_v2: bool = False
    
    # Notes
    scores: Dict[str,float] = {}
    notes: str = "Some important findings"
    LB: float = 0.0
    
    @staticmethod
    def save_config(file_name: str, output_dir: str = output_dir) -> None:
        config_dict = {}
        for key, value in vars(Config).items():
            if key.startswith('_') or isinstance(value, staticmethod):
                continue
            config_dict[key] = value
        
        out_path = f'{output_dir}{file_name}.json'
        with open(out_path, 'w') as out_file:
            json.dump(config_dict, out_file, indent=2, sort_keys=False)

In [ ]:
set_seed(Config.seed)

In [ ]:
def read_json(from_path: Path) -> dict:
    with open(from_path, 'r', encoding='utf-8') as out_file:
        return json.load(out_file)
        
def write_json(data: dict, out_path: Path) -> None:
    with open(out_path, 'w', encoding='utf-8') as out_file:
        json.dump(data, out_file, indent=2, sort_keys=True, ensure_ascii=False)

In [ ]:
xquad_guj_path = '../input/xquadgujarati/xquad.guj.fromHindi.final(Squad Version).json'
xquad_guj_dataset = chaii_dataset = datasets.load_dataset(
    'json',
    data_files=xquad_guj_path,
    field='data',
    split="train"
)
xquad_guj_dataset

In [ ]:
datasets_eng = datasets.load_dataset("squad")

In [ ]:
mlqa_dev_eng_path = '../input/mlqa-dataset-squad-format-for-question-answering/mlqa-test-en.json'
mlqa_dev_eng_dataset = datasets.load_dataset(
    'json',
    data_files=mlqa_dev_eng_path,
    field='data',
    split="train"
)
mlqa_dev_eng_dataset

In [ ]:
mlqa_test_eng_path = '../input/mlqa-dataset-squad-format-for-question-answering/mlqa-dev-en.json'
mlqa_test_eng_dataset = datasets.load_dataset(
    'json',
    data_files=mlqa_test_eng_path,
    field='data',
    split="train"
)
mlqa_test_eng_dataset

In [ ]:
xquad_eng_path = '../input/xquad-datasetsquad-format-for-question-answering/xquad-en.json'
xquad_eng_dataset = chaii_dataset = datasets.load_dataset(
    'json',
    data_files=xquad_eng_path,
    field='data',
    split="train"
)
xquad_eng_dataset

In [ ]:
final_train_dataset = datasets.concatenate_datasets([xquad_eng_dataset, mlqa_dev_eng_dataset])

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(Config.model_path)
assert isinstance(tokenizer, PreTrainedTokenizerFast)
pad_on_right = tokenizer.padding_side == "right"

In [ ]:
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=Config.max_length,
        stride=Config.doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [ ]:
tokenized_train = final_train_dataset.map(
    prepare_train_features,
    batched=True,
    remove_columns=final_train_dataset.column_names
)
validation_features = mlqa_test_eng_dataset.map(
    prepare_train_features,
    batched=True,
    remove_columns=mlqa_test_eng_dataset.column_names
)

In [ ]:
tokenized_eng_squad = datasets_eng.map(
    prepare_train_features,
    batched=True,
    remove_columns=datasets_eng['train'].column_names
)

In [ ]:
model = AutoModelForQuestionAnswering.from_pretrained(Config.model_path)

In [ ]:
from transformers import (
    AdapterConfig
)

from transformers.adapters.composition import Stack

# config = AdapterConfig.load("houlsby")

# adapter_name = model.load_adapter("qa/squad1@ukp", config=config ,model_name='roberta-base',load_as='houlsby_xlm_base')

# model.train_adapter('houlsby_xlm_base')
# model.set_active_adapters(adapter_name)


config = AdapterConfig.load("houlsby")
task_adap = model.load_adapter("qa/squad1@ukp", config=config,model_name='roberta-base',load_as='houlsby_xlm_base')
adapter_name_1 = model.load_adapter("../input/gujaratiadapterxlmr/xmlr-base-adapter/mlm",model_name='xlm-roberta-base')
adapter_name_2 = model.load_adapter("en/wiki@ukp", config=config,model_name='xlm-roberta-base')

# model.active_adapters = Stack(adapter_name_1, adapter_name_2)
model.set_active_adapters(adapter_name_2,task_adap)
model.train_adapter('houlsby_xlm_base')

In [ ]:
args = TrainingArguments(
    output_dir=Config.output_dir,
    evaluation_strategy = "epoch",
    learning_rate=Config.learning_rate,
    warmup_ratio=Config.warmup_ratio,
    gradient_accumulation_steps=Config.gradient_accumulation_steps,
    per_device_train_batch_size=Config.batch_size,
    per_device_eval_batch_size=Config.batch_size,
    num_train_epochs=Config.num_train_epochs,
    weight_decay=Config.num_train_epochs,
    seed=Config.seed
)

In [ ]:
%%capture
wandb.init(mode="disabled")
wandb.init(mode="offline")

In [ ]:
# trainer = Trainer(
#     model,
#     args,
#     train_dataset=tokenized_chaii,
#     eval_dataset=validation_features,
#     data_collator=default_data_collator,
#     tokenizer=tokenizer
# )

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_train,
    eval_dataset=validation_features,
    data_collator=default_data_collator,
    tokenizer=tokenizer
)

# trainer = Trainer(
#     model,
#     args,
#     train_dataset=tokenized_eng_squad['train'],
#     eval_dataset=tokenized_eng_squad['validation'],
#     data_collator=default_data_collator,
#     tokenizer=tokenizer
# )

In [ ]:
trainer.train()

In [ ]:
model.set_active_adapters(adapter_name_1,task_adap)


In [ ]:
def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=Config.max_length,
        stride=Config.doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [ ]:
test_features = xquad_guj_dataset.map(
    prepare_validation_features,
    batched=True,
    remove_columns=xquad_guj_dataset.column_names
)

In [ ]:
test_features

In [ ]:
raw_predictions = trainer.predict(test_features)

In [ ]:
raw_predictions

In [ ]:
test_features.set_format(type=test_features.format["type"], columns=list(test_features.features.keys()))

In [ ]:
test_features

In [ ]:
import collections
import json
import logging
import os
from typing import Optional, Tuple

import numpy as np
from tqdm.auto import tqdm


logger = logging.getLogger(__name__)

In [ ]:
def postprocess_qa_predictions(
    examples,
    features,
    predictions: Tuple[np.ndarray, np.ndarray],
    version_2_with_negative: bool = False,
    n_best_size: int = 20,
    max_answer_length: int = 30,
    null_score_diff_threshold: float = 0.0,
    output_dir: Optional[str] = None,
    prefix: Optional[str] = None,
    log_level: Optional[int] = logging.WARNING,
):
    """
    Post-processes the predictions of a question-answering model to convert them to answers that are substrings of the
    original contexts. This is the base postprocessing functions for models that only return start and end logits.
    Args:
        examples: The non-preprocessed dataset (see the main script for more information).
        features: The processed dataset (see the main script for more information).
        predictions (:obj:`Tuple[np.ndarray, np.ndarray]`):
            The predictions of the model: two arrays containing the start logits and the end logits respectively. Its
            first dimension must match the number of elements of :obj:`features`.
        version_2_with_negative (:obj:`bool`, `optional`, defaults to :obj:`False`):
            Whether or not the underlying dataset contains examples with no answers.
        n_best_size (:obj:`int`, `optional`, defaults to 20):
            The total number of n-best predictions to generate when looking for an answer.
        max_answer_length (:obj:`int`, `optional`, defaults to 30):
            The maximum length of an answer that can be generated. This is needed because the start and end predictions
            are not conditioned on one another.
        null_score_diff_threshold (:obj:`float`, `optional`, defaults to 0):
            The threshold used to select the null answer: if the best answer has a score that is less than the score of
            the null answer minus this threshold, the null answer is selected for this example (note that the score of
            the null answer for an example giving several features is the minimum of the scores for the null answer on
            each feature: all features must be aligned on the fact they `want` to predict a null answer).
            Only useful when :obj:`version_2_with_negative` is :obj:`True`.
        output_dir (:obj:`str`, `optional`):
            If provided, the dictionaries of predictions, n_best predictions (with their scores and logits) and, if
            :obj:`version_2_with_negative=True`, the dictionary of the scores differences between best and null
            answers, are saved in `output_dir`.
        prefix (:obj:`str`, `optional`):
            If provided, the dictionaries mentioned above are saved with `prefix` added to their names.
        log_level (:obj:`int`, `optional`, defaults to ``logging.WARNING``):
            ``logging`` log level (e.g., ``logging.WARNING``)
    """
    assert len(predictions) == 2, "`predictions` should be a tuple with two elements (start_logits, end_logits)."
    all_start_logits, all_end_logits = predictions

    assert len(predictions[0]) == len(features), f"Got {len(predictions[0])} predictions and {len(features)} features."

    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    all_predictions = collections.OrderedDict()
    all_nbest_json = collections.OrderedDict()
    if version_2_with_negative:
        scores_diff_json = collections.OrderedDict()

    # Logging.
    logger.setLevel(log_level)
    logger.info(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_prediction = None
        prelim_predictions = []

        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]
            # Optional `token_is_max_context`, if provided we will remove answers that do not have the maximum context
            # available in the current feature.
            token_is_max_context = features[feature_index].get("token_is_max_context", None)

            # Update minimum null prediction.
            feature_null_score = start_logits[0] + end_logits[0]
            if min_null_prediction is None or min_null_prediction["score"] > feature_null_score:
                min_null_prediction = {
                    "offsets": (0, 0),
                    "score": feature_null_score,
                    "start_logit": start_logits[0],
                    "end_logit": end_logits[0],
                }

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue
                    # Don't consider answer that don't have the maximum context available (if such information is
                    # provided).
                    if token_is_max_context is not None and not token_is_max_context.get(str(start_index), False):
                        continue
                    prelim_predictions.append(
                        {
                            "offsets": (offset_mapping[start_index][0], offset_mapping[end_index][1]),
                            "score": start_logits[start_index] + end_logits[end_index],
                            "start_logit": start_logits[start_index],
                            "end_logit": end_logits[end_index],
                        }
                    )
        if version_2_with_negative:
            # Add the minimum null prediction
            prelim_predictions.append(min_null_prediction)
            null_score = min_null_prediction["score"]

        # Only keep the best `n_best_size` predictions.
        predictions = sorted(prelim_predictions, key=lambda x: x["score"], reverse=True)[:n_best_size]

        # Add back the minimum null prediction if it was removed because of its low score.
        if version_2_with_negative and not any(p["offsets"] == (0, 0) for p in predictions):
            predictions.append(min_null_prediction)

        # Use the offsets to gather the answer text in the original context.
        context = example["context"]
        for pred in predictions:
            offsets = pred.pop("offsets")
            pred["text"] = context[offsets[0] : offsets[1]]

        # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
        # failure.
        if len(predictions) == 0 or (len(predictions) == 1 and predictions[0]["text"] == ""):
            predictions.insert(0, {"text": "empty", "start_logit": 0.0, "end_logit": 0.0, "score": 0.0})

        # Compute the softmax of all scores (we do it with numpy to stay independent from torch/tf in this file, using
        # the LogSumExp trick).
        scores = np.array([pred.pop("score") for pred in predictions])
        exp_scores = np.exp(scores - np.max(scores))
        probs = exp_scores / exp_scores.sum()

        # Include the probabilities in our predictions.
        for prob, pred in zip(probs, predictions):
            pred["probability"] = prob

        # Pick the best prediction. If the null answer is not possible, this is easy.
        if not version_2_with_negative:
            all_predictions[example["id"]] = predictions[0]["text"]
        else:
            # Otherwise we first need to find the best non-empty prediction.
            i = 0
            while predictions[i]["text"] == "":
                i += 1
            best_non_null_pred = predictions[i]

            # Then we compare to the null prediction using the threshold.
            score_diff = null_score - best_non_null_pred["start_logit"] - best_non_null_pred["end_logit"]
            scores_diff_json[example["id"]] = float(score_diff)  # To be JSON-serializable.
            if score_diff > null_score_diff_threshold:
                all_predictions[example["id"]] = ""
            else:
                all_predictions[example["id"]] = best_non_null_pred["text"]

        # Make `predictions` JSON-serializable by casting np.float back to float.
        all_nbest_json[example["id"]] = [
            {k: (float(v) if isinstance(v, (np.float16, np.float32, np.float64)) else v) for k, v in pred.items()}
            for pred in predictions
        ]

    # If we have an output_dir, let's save all those dicts.
    if output_dir is not None:
        assert os.path.isdir(output_dir), f"{output_dir} is not a directory."

        prediction_file = os.path.join(
            output_dir, "predictions.json" if prefix is None else f"{prefix}_predictions.json"
        )
        nbest_file = os.path.join(
            output_dir, "nbest_predictions.json" if prefix is None else f"{prefix}_nbest_predictions.json"
        )
        if version_2_with_negative:
            null_odds_file = os.path.join(
                output_dir, "null_odds.json" if prefix is None else f"{prefix}_null_odds.json"
            )

        logger.info(f"Saving predictions to {prediction_file}.")
        with open(prediction_file, "w") as writer:
            writer.write(json.dumps(all_predictions, indent=4) + "\n")
        logger.info(f"Saving nbest_preds to {nbest_file}.")
        with open(nbest_file, "w") as writer:
            writer.write(json.dumps(all_nbest_json, indent=4) + "\n")
        if version_2_with_negative:
            logger.info(f"Saving null_odds to {null_odds_file}.")
            with open(null_odds_file, "w") as writer:
                writer.write(json.dumps(scores_diff_json, indent=4) + "\n")

    return all_predictions

In [ ]:
from tqdm.auto import tqdm
import gc

from IPython.display import FileLink

In [ ]:
final_predictions = postprocess_qa_predictions(xquad_guj_dataset, test_features, raw_predictions.predictions,output_dir='./')

In [ ]:
# final_predictions.items()

In [ ]:
metric = datasets.load_metric("squad")

In [ ]:
formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
references = [{"id": ex["id"], "answers": ex["answers"]} for ex in xquad_guj_dataset]

In [ ]:
formatted_predictions[-2:]

In [ ]:
references[-2:]

In [ ]:
scores = metric.compute(predictions=formatted_predictions, references=references)
scores

In [ ]:
with open('./real_ans.json', 'w') as f:
    json.dump(references, f)

In [ ]:
with open('./formatted_predictions.json', 'w') as f:
    json.dump(formatted_predictions, f)

In [ ]:
scores["description"] = 'Used Full dataset to test the basic trained model gujarati bert-base-multilingual-cased'
# scores['length train'] = len(final_train_dataset)
scores['length test'] = len(xquad_guj_dataset)

In [ ]:
def jaccard(row: pd.Series) -> float: 
    str1 = row[0]
    str2 = row[1]
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [ ]:
!pip install jiwer

In [ ]:
import jiwer

In [ ]:
wer_metric = datasets.load_metric("wer")

In [ ]:
formatted_predictions[:2]

In [ ]:
references[:2]

In [ ]:
eval_df = pd.DataFrame(references)
eval_df['answers'] = eval_df.answers.apply(lambda x: x['text'][0])
eval_df['predictions'] = eval_df['id'].apply(lambda x: final_predictions[x])
eval_df['jaccard'] = eval_df[['answers', 'predictions']].apply(jaccard, axis=1)
eval_df['wer'] = eval_df[['predictions', 'answers']].apply(lambda x: wer_metric.compute(predictions=x[[0]], references=[x[1]]), axis=1)
eval_df.head()

In [ ]:
eval_df.shape

In [ ]:
eval_df.describe()

eval_df.to_json('./predictions_with_scores.json',orient='table')

In [ ]:
eval_data = read_json(r'./predictions_with_scores.json')

eval_data['data'][:4]

In [ ]:
jaccard_wer_scores = {'jaccard': eval_df.jaccard.mean(), 'wer': eval_df.wer.mean()}
scores.update(jaccard_wer_scores)
jaccard_wer_scores

In [ ]:
training_output = f"{Config.model_name}-finetuned-{Config.version}"
trainer.save_model(training_output)

In [ ]:
scores

In [ ]:
with open('./all_results.json', 'w') as f:
    json.dump(scores, f)

In [ ]:
notes = 'bert-base-multilingual-cased-hindi-A,  add external data. Change max len to 256.'
Config.notes = notes
Config.scores = scores
Config.save_config(training_output.replace('-', '_'))

In [ ]:
# %cd ./
%ls

# Download 

**NOTE:** This is training notebook will work only in an online mode.  You do not allow internet access to submit your prediction. While you can modify this code to use it without access to the internet and send your predictions directly,  the more convenient way is to create an inference notebook without all this training staff, but only applying the model on a test dataset. To do this, you need to load the directory that you created in the previous step when calling

```python
trainer.save_model(training_output)
```

Then you need to create a new (or update an existing) Kaggle dataset, load the whole thing, and connect the dataset to your inference kernel. Of course, if the loop is worth it, otherwise just download the experiment config and keep looking for a solution :)

In [ ]:
!zip -r  bert-base-multilingual-cased-finetuned-hindi-A.zip  ./bert-base-multilingual-cased-finetuned-hindi-A

In [ ]:
!ls


In [ ]:
from IPython.display import FileLink

In [ ]:
FileLink(r'./bert-base-multilingual-cased-finetuned-hindi-A.zip')

In [ ]:
FileLink(r'./bert_base_multilingual_cased_finetuned_hindi_A.json')

In [ ]:
FileLink(r'./all_results.json')

In [ ]:
FileLink(r'./predictions_with_scores.json')

In [ ]:
FileLink(r'./predictions.json')

In [ ]:
FileLink(r'./nbest_predictions.json')

In [ ]:
FileLink(r'./real_ans.json')

In [ ]:
FileLink(r'./formatted_predictions.json')